## 전월세가 예측 데이터 전처리

1. 전처리 목표: 모든 데이터 병합 후 코스피 금리 전월세전환 비율 추가

  - 패키지 설치


---


2. 데이터 병합
  - 오피스텔 데이터 병합
  - 아파트 데이터 병합
  - 다세대연립 데이터 병합
  - 실거래 종합 데이터 생성


---


3. 변수 추가
  - 추가변수 로드 (코스피, 전월세전환비율, 금리)
  - 코스피 데이터 처리 및 종합 데이터에 변수 추가
  - 금리 데이터 처리 및 종합 데이터에 변수 추가
  - 전월세전환 비율 데이터 처리 및 종합 데이터에 변수 추가


---


4. 올바른 도로명 주소 변경
  - 도로명 컬럼에서 빈 문자열 제거 및 전체 주소 완성

---

5. 데이터 저장
  - 월세를 전세로 전환 시킨 데이터 저장
    1. 월세인 행의 경우 계약년월을 기준으로 전월세변환율을 적용하여 변환
    2. 실수 -> 정수 변환(금액의 소수점은 없음)
    3. 전월세변환 데이터 저장: change_data.csv
  - 4 단계에서 추가 처리 없이 저장: full_data.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 필요한 패키지 설치

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

print("Okay, let's get started!")

Okay, let's get started!


#### 오피스텔 데이터 병합(MERGE)

In [ ]:
# Load the data
op1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_1805.csv",encoding='cp949', skiprows=15)
op2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_1905.csv',encoding='cp949', skiprows=15)
op3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2005.csv',encoding='cp949', skiprows=15)
op4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2105.csv',encoding='cp949', skiprows=15)
op5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2205.csv',encoding='cp949', skiprows=15)
op6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2305.csv',encoding='cp949', skiprows=15)
op7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2405.csv',encoding='cp949', skiprows=15)


<ipython-input-4-6a00cef6051b>:5: DtypeWarning: Columns (11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  op4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2105.csv',encoding='cp949', skiprows=15)
<ipython-input-4-6a00cef6051b>:6: DtypeWarning: Columns (11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  op5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2205.csv',encoding='cp949', skiprows=15)
<ipython-input-4-6a00cef6051b>:7: DtypeWarning: Columns (11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  op6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/op/op_2305.csv',encoding='cp949', skiprows=15)


In [ ]:
# 데이터 확인
column1 = op1.columns
column2 = op2.columns
column3 = op3.columns
column4 = op4.columns
column5 = op5.columns
column6 = op6.columns
column7 = op7.columns

overlap = set(column1) & set(column2) & set(column3) & set(column4) & set(column5) & set(column6) & set(column7)
print(overlap)
print(len(overlap))


{'층', 'NO', '월세금(만원)', '계약일', '계약년월', '도로명', '종전계약 보증금(만원)', '단지명', '계약기간', '건축년도', '주택유형', '종전계약 월세(만원)', '계약구분', '갱신요구권 사용', '전월세구분', '본번', '시군구', '부번', '번지', '보증금(만원)', '전용면적(㎡)'}
21


In [ ]:
# 데이터 합치기
op_dat = pd.concat([op1, op2], axis=0)
op_dat = pd.concat([op_dat, op3], axis=0)
op_dat = pd.concat([op_dat, op4], axis=0)
op_dat = pd.concat([op_dat, op5], axis=0)
op_dat = pd.concat([op_dat, op6], axis=0)
op_dat = pd.concat([op_dat, op7], axis=0)


In [ ]:
print(op_dat.shape)

(414008, 21)


In [ ]:
op1.dtypes

,0
NO,int64
시군구,object
번지,object
본번,int64
부번,int64
단지명,object
전월세구분,object
전용면적(㎡),float64
계약년월,int64
계약일,int64


#### 아파트 데이터 병합

In [ ]:
apt1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_1805.csv", skiprows=15, encoding='cp949')
apt2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_1905.csv", skiprows=15, encoding='cp949')
apt3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2005.csv", skiprows=15, encoding='cp949')
apt4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2105.csv", skiprows=15, encoding='cp949')
apt5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2205.csv", skiprows=15, encoding='cp949')
apt6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2305.csv", skiprows=15, encoding='cp949')
apt7 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2405.csv", skiprows=15, encoding='cp949')


<ipython-input-9-a8ae2a96ffb7>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  apt1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_1805.csv", skiprows=15, encoding='cp949')
<ipython-input-9-a8ae2a96ffb7>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  apt2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_1905.csv", skiprows=15, encoding='cp949')
<ipython-input-9-a8ae2a96ffb7>:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  apt4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/apt/apt_2105.csv", skiprows=15, encoding='cp949')
<ipython-input-9-a8ae2a96ffb7>:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  apt5 = pd.read_csv("/content/drive/MyDrive/Colab N

In [ ]:
# 중복 컬럼 확인
aptcol1 = apt1.columns
aptcol2 = apt2.columns
aptcol3 = apt3.columns
aptcol4 = apt4.columns
aptcol5 = apt5.columns
aptcol6 = apt6.columns
aptcol7 = apt7.columns

apt_overlap = set(aptcol1) & set(aptcol2) & set(aptcol3) & set(aptcol4) & set(aptcol5) & set(aptcol6) & set(aptcol7)
print(apt_overlap)
print(len(apt_overlap))


{'층', 'NO', '월세금(만원)', '계약일', '계약년월', '도로명', '종전계약 보증금(만원)', '단지명', '계약기간', '건축년도', '주택유형', '종전계약 월세(만원)', '계약구분', '갱신요구권 사용', '전월세구분', '본번', '시군구', '부번', '번지', '보증금(만원)', '전용면적(㎡)'}
21


In [ ]:
apt_dat = pd.concat([apt1, apt2], axis=0)
apt_dat = pd.concat([apt_dat, apt3], axis=0)
apt_dat = pd.concat([apt_dat, apt4], axis=0)
apt_dat = pd.concat([apt_dat, apt5], axis=0)
apt_dat = pd.concat([apt_dat, apt6], axis=0)
apt_dat = pd.concat([apt_dat, apt7], axis=0)

In [ ]:
apt_dat.shape

(1461237, 21)

#### 다세대연립 데이터 병합

In [ ]:
multi1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_1805.csv", skiprows=15, encoding='cp949')
multi2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_1905.csv", skiprows=15, encoding='cp949')
multi3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2005.csv", skiprows=15, encoding='cp949')
multi4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2105.csv", skiprows=15, encoding='cp949')
multi5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2205.csv", skiprows=15, encoding='cp949')
multi6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2305.csv", skiprows=15, encoding='cp949')
multi7 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2405.csv", skiprows=15, encoding='cp949')

<ipython-input-13-5f7d1fa37570>:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  multi1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_1805.csv", skiprows=15, encoding='cp949')
<ipython-input-13-5f7d1fa37570>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  multi2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_1905.csv", skiprows=15, encoding='cp949')
<ipython-input-13-5f7d1fa37570>:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  multi4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가/multi/multi_2105.csv", skiprows=15, encoding='cp949')
<ipython-input-13-5f7d1fa37570>:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  multi5 = pd.read_csv("/conte

In [ ]:
# 중복 컬럼 확인

multicol1 = multi1.columns
multicol2 = multi2.columns
multicol3 = multi3.columns
multicol4 = multi4.columns
multicol5 = multi5.columns
multicol6 = multi6.columns
multicol7 = multi7.columns
multi_overlap = set(multicol1) & set(multicol2) & set(multicol3) & set(multicol4) & set(multicol5) & set(multicol6) & set(multicol7)
print(multi_overlap)
print(apt_overlap & multi_overlap & overlap)

{'층', 'NO', '월세금(만원)', '계약일', '계약년월', '도로명', '종전계약 보증금(만원)', '계약기간', '건축년도', '주택유형', '종전계약 월세(만원)', '계약구분', '갱신요구권 사용', '전월세구분', '본번', '시군구', '부번', '번지', '보증금(만원)', '전용면적(㎡)'}
{'층', 'NO', '월세금(만원)', '계약일', '계약년월', '도로명', '종전계약 보증금(만원)', '계약기간', '건축년도', '주택유형', '종전계약 월세(만원)', '계약구분', '갱신요구권 사용', '전월세구분', '본번', '시군구', '부번', '번지', '보증금(만원)', '전용면적(㎡)'}


In [ ]:
# 데이터 머지
multi_dat = pd.concat([multi1, multi2], axis=0)
multi_dat = pd.concat([multi_dat, multi3], axis=0)
multi_dat = pd.concat([multi_dat, multi4], axis=0)
multi_dat = pd.concat([multi_dat, multi5], axis=0)
multi_dat = pd.concat([multi_dat, multi6], axis=0)
multi_dat = pd.concat([multi_dat, multi7], axis=0)

In [ ]:
multi_dat.shape

(1420509, 22)

In [ ]:
new = multi_dat.columns
a = 1
for i in new:

  if i in apt_overlap:
    print("True", a)
    a += 1
  else:
    print("False", a)
    a += 1

new[5]

True 1
True 2
True 3
True 4
True 5
False 6
True 7
True 8
True 9
True 10
True 11
True 12
True 13
True 14
True 15
True 16
True 17
True 18
True 19
True 20
True 21
True 22


'건물명'

In [ ]:
multi_dat['건물명'].isnull().sum()


1290170

#### 실거래 종합 데이터 생성

In [ ]:
nece = list(apt_overlap)
multi_dat = multi_dat[nece]
print(multi_dat.columns, len(multi_dat.columns))


Index(['층', 'NO', '월세금(만원)', '계약일', '계약년월', '도로명', '종전계약 보증금(만원)', '단지명',
       '계약기간', '건축년도', '주택유형', '종전계약 월세(만원)', '계약구분', '갱신요구권 사용', '전월세구분',
       '본번', '시군구', '부번', '번지', '보증금(만원)', '전용면적(㎡)'],
      dtype='object') 21


In [ ]:
total_dat = pd.concat([op_dat, apt_dat, multi_dat], axis=0)

In [ ]:
total_dat.shape

(3295754, 21)

In [ ]:
total_dat.drop(['NO', '번지', '본번', '부번'], axis=1, inplace=True)
total_dat.shape


(3295754, 17)

In [ ]:
total_dat.columns

Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)',
       '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)',
       '종전계약 월세(만원)', '주택유형'],
      dtype='object')

In [ ]:
total_dat.dtypes

,0
시군구,object
단지명,object
전월세구분,object
전용면적(㎡),float64
계약년월,int64
계약일,int64
보증금(만원),object
월세금(만원),object
층,int64
건축년도,float64


In [ ]:
# 계약년월을 수치형이 아닌 날짜 데이터로 변환
total_dat['계약년월']

,계약년월
0,201904
1,201904
2,201904
3,201904
4,201904
...,...
115789,202405
115790,202405
115791,202405
115792,202405


In [ ]:
total_dat['계약년월'] = total_dat['계약년월'].dropna().astype(int).astype(str).apply(lambda x: f"{x[:4]}-{x[4:]}")
total_dat['계약년월']

,계약년월
0,2019-04
1,2019-04
2,2019-04
3,2019-04
4,2019-04
...,...
115789,2024-05
115790,2024-05
115791,2024-05
115792,2024-05


In [ ]:
total_dat['계약년월'].max()
# 추가 변수에 2024-11 부터는 존재 하지 않으니 제거

'2025-01'

In [ ]:
# 제거할 값 리스트
remove_dates = ["2024-11", "2024-12", "2025-01"]

# 해당 값이 포함된 행 제거
total_dat = total_dat[~total_dat['계약년월'].isin(remove_dates)]

# 인덱스 리셋 (필요하면)
total_dat = total_dat.reset_index(drop=True)

print(total_dat.shape, "\n", total_dat['계약년월'].max(),"\n", total_dat.describe())


(3272282, 17) 
 2024-10 
             전용면적(㎡)           계약일             층          건축년도
count  3.272282e+06  3.272282e+06  3.272282e+06  3.267243e+06
mean   6.374364e+01  1.600116e+01  9.057175e+00  2.005194e+03
std    3.122001e+01  8.581156e+00  6.180304e+00  1.213499e+01
min    3.400000e+00  1.000000e+00 -4.000000e+00  1.956000e+03
25%    3.986000e+01  9.000000e+00  4.000000e+00  1.997000e+03
50%    5.994000e+01  1.600000e+01  8.000000e+00  2.006000e+03
75%    8.487000e+01  2.300000e+01  1.300000e+01  2.016000e+03
max    4.451200e+02  3.100000e+01  6.900000e+01  2.025000e+03


#### 추가변수 로드

In [ ]:
kospi = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가_참고/코스피지수_시가총액.csv")
pp_dat = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가_참고/전월세전환율.csv", encoding = 'cp949')
rate = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/전월세가_참고/한국은행 기준금리 및 여수신금리_28213932.csv")

#### kospi 처리

In [ ]:
# 데이터 확인
kospi

,통계표,계정항목,단위,변환,2018.5.1,2018.6.1,2018.7.1,2018.8.1,2018.9.1,2018.10.1,...,2024.1.1,2024.2.1,2024.3.1,2024.4.1,2024.5.1,2024.6.1,2024.7.1,2024.8.1,2024.9.1,2024.10.1
0,"1.5.1.2. 주식시장(월,년)",KOSPI_시가총액,천원,원자료,"1,619,387,832,449","1,554,717,340,202","1,536,034,531,292","1,554,873,150,916","1,567,600,410,138","1,361,478,009,940",...,"2,030,032,668,711","2,154,501,763,496","2,240,537,707,931","2,193,843,858,061","2,151,017,465,412","2,283,888,693,661","2,262,832,341,049","2,187,461,755,865","2,115,975,531,403","2,083,807,734,486"
1,"1.5.1.2. 주식시장(월,년)",KOSPI_종가,1980.01.04=100,원자료,"2,423.01","2,326.13","2,295.26","2,322.88","2,343.07","2,029.69",...,"2,497.09","2,642.36","2,746.63","2,692.06","2,636.52","2,797.82","2,770.69","2,674.31","2,593.27","2,556.15"


In [ ]:
kospi.dtypes

,0
통계표,object
계정항목,object
단위,object
변환,object
2018.5.1,object
...,...
2024.6.1,object
2024.7.1,object
2024.8.1,object
2024.9.1,object


In [ ]:
date = kospi.columns
date = date[4:]
last = kospi[date].iloc[-1]
last
date = list(date)
last = list(last)
for i in range(len(date)):
  temp = date[i].split(".")
  if len(temp[1]) == 2:
    date[i] = temp[0]+"-"+temp[1]

  else:
    date[i] = temp[0]+"-0"+temp[1]


for i in range(len(last)):
  last[i] = last[i].replace(",","")
  last[i] = float(last[i])


In [ ]:
kospi_df = pd.DataFrame({'date': date, 'index': last})


kospi_df.dtypes

,0
date,object
index,float64


In [ ]:
kospi_df.iloc[72]

,72
date,2024-05
index,2636.52


In [ ]:

total_dat
# 'total_dat'와 'kospi_df'를 '계약년월'과 'date' 기준으로 병합
merged_df = pd.merge(total_dat, kospi_df, left_on='계약년월', right_on='date', how='left')
merged_df = merged_df.drop(columns=['date'])  # 'date' 컬럼 제거
merged_df

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,"18,000",0,3,2018.0,상원6나길 24,-,-,-,NaN,NaN,오피스텔,2203.59
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,"23,000",0,4,2019.0,불광로14길 4,-,-,-,NaN,NaN,오피스텔,2203.59
2,서울특별시 마포구 상암동,사보이시티디엠씨,전세,21.6500,2019-04,30,"15,500",0,10,2016.0,,-,-,-,NaN,NaN,오피스텔,2203.59
3,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,"11,000",0,4,1999.0,능동로19길 47,-,-,-,NaN,NaN,오피스텔,2203.59
4,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,"1,000",50,4,2005.0,도봉로 352,-,-,-,NaN,NaN,오피스텔,2203.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272277,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,"60,900",25,5,2013.0,위례광장로 121,202405~202605,갱신,-,"60,900",0,아파트,2636.52
3272278,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,"92,500",0,6,2015.0,위례광장로 185,202407~202607,신규,-,NaN,NaN,아파트,2636.52
3272279,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,"5,000",179,19,2008.0,올림픽로 135,202406~202606,갱신,사용,"5,000",170,아파트,2636.52
3272280,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,"100,000",0,2,2008.0,올림픽로 99,202407~202607,신규,-,NaN,NaN,아파트,2636.52


In [ ]:
merged_df.columns

Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)',
       '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)',
       '종전계약 월세(만원)', '주택유형', 'index'],
      dtype='object')

In [ ]:
merged_df.dtypes

,0
시군구,object
단지명,object
전월세구분,object
전용면적(㎡),float64
계약년월,object
계약일,int64
보증금(만원),object
월세금(만원),object
층,int64
건축년도,float64


In [ ]:
merged_df.describe()

,전용면적(㎡),계약일,층,건축년도,index
count,3.272282e+06,3.272282e+06,3.272282e+06,3.267243e+06,3.272282e+06
mean,6.374364e+01,1.600116e+01,9.057175e+00,2.005194e+03,2.530236e+03
std,3.122001e+01,8.581156e+00,6.180304e+00,1.213499e+01,3.380031e+02
min,3.400000e+00,1.000000e+00,-4.000000e+00,1.956000e+03,1.754640e+03
25%,3.986000e+01,9.000000e+00,4.000000e+00,1.997000e+03,2.277990e+03
50%,5.994000e+01,1.600000e+01,8.000000e+00,2.006000e+03,2.556150e+03
75%,8.487000e+01,2.300000e+01,1.300000e+01,2.016000e+03,2.699180e+03
max,4.451200e+02,3.100000e+01,6.900000e+01,2.025000e+03,3.296680e+03


In [ ]:
merged_df.isnull().sum()

,0
시군구,0
단지명,106869
전월세구분,2
전용면적(㎡),0
계약년월,0
계약일,0
보증금(만원),0
월세금(만원),0
층,0
건축년도,5039


In [ ]:
# # 결측 데이터 원인
# missing_dates = merged_df[merged_df['index'].isna()]['계약년월']
# print(missing_dates)
# # 특정 구간임을 확인 비교를 위해 수치형으로 변환
# for i in range(len(missing_dates)):
#   missing_dates.iloc[i] = missing_dates.iloc[i].replace("-","")
#   missing_dates.iloc[i] = int(missing_dates.iloc[i])


In [ ]:
# print(f"min = {missing_dates.min()}, max = {missing_dates.max()}")
# missing_dates.value_counts()

#### 기준금리 처리

In [ ]:
rate.head()

,통계표,계정항목,단위,변환,2018.5.1,2018.6.1,2018.7.1,2018.8.1,2018.9.1,2018.10.1,...,2024.1.1,2024.2.1,2024.3.1,2024.4.1,2024.5.1,2024.6.1,2024.7.1,2024.8.1,2024.9.1,2024.10.1
0,1.3.1. 한국은행 기준금리 및 여수신금리,한국은행 기준금리,연%,원자료,1.5,1.5,1.5,1.5,1.5,1.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.25


In [ ]:
rate_date = rate.columns
rate_date = rate_date[4:]
rate_date

Index(['2018.5.1', '2018.6.1', '2018.7.1', '2018.8.1', '2018.9.1', '2018.10.1',
       '2018.11.1', '2018.12.1', '2019.1.1', '2019.2.1', '2019.3.1',
       '2019.4.1', '2019.5.1', '2019.6.1', '2019.7.1', '2019.8.1', '2019.9.1',
       '2019.10.1', '2019.11.1', '2019.12.1', '2020.1.1', '2020.2.1',
       '2020.3.1', '2020.4.1', '2020.5.1', '2020.6.1', '2020.7.1', '2020.8.1',
       '2020.9.1', '2020.10.1', '2020.11.1', '2020.12.1', '2021.1.1',
       '2021.2.1', '2021.3.1', '2021.4.1', '2021.5.1', '2021.6.1', '2021.7.1',
       '2021.8.1', '2021.9.1', '2021.10.1', '2021.11.1', '2021.12.1',
       '2022.1.1', '2022.2.1', '2022.3.1', '2022.4.1', '2022.5.1', '2022.6.1',
       '2022.7.1', '2022.8.1', '2022.9.1', '2022.10.1', '2022.11.1',
       '2022.12.1', '2023.1.1', '2023.2.1', '2023.3.1', '2023.4.1', '2023.5.1',
       '2023.6.1', '2023.7.1', '2023.8.1', '2023.9.1', '2023.10.1',
       '2023.11.1', '2023.12.1', '2024.1.1', '2024.2.1', '2024.3.1',
       '2024.4.1', '2024.5.1', '2024.6.

In [ ]:
rate_rate = rate[rate_date].iloc[-1]
rate_rate

,0
2018.5.1,1.50
2018.6.1,1.50
2018.7.1,1.50
2018.8.1,1.50
2018.9.1,1.50
...,...
2024.6.1,3.50
2024.7.1,3.50
2024.8.1,3.50
2024.9.1,3.50


In [ ]:
# 날짜 형식 변환을 위한 리스트
rate_date = list(rate_date)
rate_rate = list(rate_rate)
rate_date

['2018.5.1',
 '2018.6.1',
 '2018.7.1',
 '2018.8.1',
 '2018.9.1',
 '2018.10.1',
 '2018.11.1',
 '2018.12.1',
 '2019.1.1',
 '2019.2.1',
 '2019.3.1',
 '2019.4.1',
 '2019.5.1',
 '2019.6.1',
 '2019.7.1',
 '2019.8.1',
 '2019.9.1',
 '2019.10.1',
 '2019.11.1',
 '2019.12.1',
 '2020.1.1',
 '2020.2.1',
 '2020.3.1',
 '2020.4.1',
 '2020.5.1',
 '2020.6.1',
 '2020.7.1',
 '2020.8.1',
 '2020.9.1',
 '2020.10.1',
 '2020.11.1',
 '2020.12.1',
 '2021.1.1',
 '2021.2.1',
 '2021.3.1',
 '2021.4.1',
 '2021.5.1',
 '2021.6.1',
 '2021.7.1',
 '2021.8.1',
 '2021.9.1',
 '2021.10.1',
 '2021.11.1',
 '2021.12.1',
 '2022.1.1',
 '2022.2.1',
 '2022.3.1',
 '2022.4.1',
 '2022.5.1',
 '2022.6.1',
 '2022.7.1',
 '2022.8.1',
 '2022.9.1',
 '2022.10.1',
 '2022.11.1',
 '2022.12.1',
 '2023.1.1',
 '2023.2.1',
 '2023.3.1',
 '2023.4.1',
 '2023.5.1',
 '2023.6.1',
 '2023.7.1',
 '2023.8.1',
 '2023.9.1',
 '2023.10.1',
 '2023.11.1',
 '2023.12.1',
 '2024.1.1',
 '2024.2.1',
 '2024.3.1',
 '2024.4.1',
 '2024.5.1',
 '2024.6.1',
 '2024.7.1',
 '2024.

In [ ]:
for i in range(len(rate_date)):
  temp = rate_date[i].split(".")
  if len(temp[1]) == 2:
    rate_date[i] = temp[0]+"-"+temp[1]

  else:
    rate_date[i] = temp[0]+"-0"+temp[1]


In [ ]:
rate_df = pd.DataFrame({'date': rate_date, 'rate': rate_rate})
rate_df

,date,rate
0,2018-05,1.50
1,2018-06,1.50
2,2018-07,1.50
3,2018-08,1.50
4,2018-09,1.50
...,...,...
73,2024-06,3.50
74,2024-07,3.50
75,2024-08,3.50
76,2024-09,3.50


In [ ]:
merged_rate_df = pd.merge(merged_df, rate_df, left_on='계약년월', right_on='date', how='left')

merged_rate_df = merged_rate_df.drop(columns=['date'])  # 'date' 컬럼 제거
merged_rate_df

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,"18,000",0,3,2018.0,상원6나길 24,-,-,-,NaN,NaN,오피스텔,2203.59,1.75
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,"23,000",0,4,2019.0,불광로14길 4,-,-,-,NaN,NaN,오피스텔,2203.59,1.75
2,서울특별시 마포구 상암동,사보이시티디엠씨,전세,21.6500,2019-04,30,"15,500",0,10,2016.0,,-,-,-,NaN,NaN,오피스텔,2203.59,1.75
3,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,"11,000",0,4,1999.0,능동로19길 47,-,-,-,NaN,NaN,오피스텔,2203.59,1.75
4,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,"1,000",50,4,2005.0,도봉로 352,-,-,-,NaN,NaN,오피스텔,2203.59,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272277,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,"60,900",25,5,2013.0,위례광장로 121,202405~202605,갱신,-,"60,900",0,아파트,2636.52,3.50
3272278,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,"92,500",0,6,2015.0,위례광장로 185,202407~202607,신규,-,NaN,NaN,아파트,2636.52,3.50
3272279,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,"5,000",179,19,2008.0,올림픽로 135,202406~202606,갱신,사용,"5,000",170,아파트,2636.52,3.50
3272280,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,"100,000",0,2,2008.0,올림픽로 99,202407~202607,신규,-,NaN,NaN,아파트,2636.52,3.50


In [ ]:
merged_rate_df.columns


Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)',
       '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)',
       '종전계약 월세(만원)', '주택유형', 'index', 'rate'],
      dtype='object')

In [ ]:
merged_rate_df.isnull().sum()

,0
시군구,0
단지명,106869
전월세구분,2
전용면적(㎡),0
계약년월,0
계약일,0
보증금(만원),0
월세금(만원),0
층,0
건축년도,5039


#### 전월세전환율 처리

In [ ]:
pp_dat.head()


,주택유형별,지역별,항목,단위,2018.05 월,2018.06 월,2018.07 월,2018.08 월,2018.09 월,2018.10 월,...,2024.02 월,2024.03 월,2024.04 월,2024.05 월,2024.06 월,2024.07 월,2024.08 월,2024.09 월,2024.10 월,Unnamed: 82
0,종합,전국,지역별 전월세전환율,NaN,6.359348,6.343863,6.313884,6.268846,6.227109,6.158501,...,6.162903,6.102222,6.003090,5.906597,5.815386,5.787400,5.813495,5.905105,6.020348,NaN
1,종합,수도권,지역별 전월세전환율,NaN,5.842298,5.843955,5.830728,5.813151,5.795628,5.736519,...,5.800557,5.751389,5.673478,5.609597,5.542936,5.533393,5.542482,5.632248,5.726139,NaN
2,종합,지방,지역별 전월세전환율,NaN,7.557447,7.538459,7.490837,7.405982,7.340357,7.254985,...,6.852179,6.797722,6.648130,6.518034,6.377502,6.322695,6.396248,6.502396,6.667701,NaN
3,종합,5대광역시,지역별 전월세전환율,NaN,7.220088,7.207055,7.198285,7.078280,7.030456,6.932411,...,6.387502,6.374464,6.274036,6.183581,6.096361,6.016017,6.004395,6.073533,6.189795,NaN
4,종합,8개도,지역별 전월세전환율,NaN,7.990598,7.934521,7.874187,7.799588,7.728622,7.658037,...,7.417975,7.320583,7.122017,6.954082,6.751300,6.698319,6.829644,6.993354,7.188155,NaN


In [ ]:
pp_dat.shape

(262, 83)

In [ ]:
pp_date = pp_dat.columns
pp_date = pp_date[4:-1]

pp_date = list(pp_date)
for i in range(len(pp_date)):
  pp_date[i] = pp_date[i].replace(".","-")
  pp_date[i] = pp_date[i][:7]
pp_date = pd.Series(pp_date)
pp_date

,0
0,2018-05
1,2018-06
2,2018-07
3,2018-08
4,2018-09
...,...
73,2024-06
74,2024-07
75,2024-08
76,2024-09


In [ ]:
# 서울인 데이터 행을 확인
pp_lo = list(pp_dat['지역별'])
if '서울' in pp_lo:
  print(pp_lo.index('서울'))
pp_lo.count('서울')

5


4

In [ ]:
# 전월세 데이터에 지역이 서울만 있음을 확인
local = list(total_dat["시군구"])
for i in range(len(local)):
  local[i] = local[i].split()[0]

local = pd.Series(local)
local.describe()

,0
count,3272282
unique,1
top,서울특별시
freq,3272282


In [ ]:
seoul_data = pp_dat[pp_dat['지역별'] == '서울']
seoul_data.drop(['지역별', '항목', '단위'], axis=1, inplace=True)


<ipython-input-56-ae67317ec3d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_data.drop(['지역별', '항목', '단위'], axis=1, inplace=True)


In [ ]:
seoul_data = seoul_data[seoul_data['주택유형별'] == '종합'].reset_index(drop=True)


In [ ]:
seoul_data

,주택유형별,2018.05 월,2018.06 월,2018.07 월,2018.08 월,2018.09 월,2018.10 월,2018.11 월,2018.12 월,2019.01 월,...,2024.02 월,2024.03 월,2024.04 월,2024.05 월,2024.06 월,2024.07 월,2024.08 월,2024.09 월,2024.10 월,Unnamed: 82
0,종합,5.417111,5.440604,5.426804,5.387159,5.360421,5.299121,5.256796,5.226404,5.213385,...,5.241678,5.212908,5.162828,5.091858,5.017016,4.97483,4.930091,5.023079,5.082617,NaN


In [ ]:
seoul_data = np.array(seoul_data.iloc[0])
seoul_data = pd.Series(seoul_data[1:-1])
seoul_data

,0
0,5.417111
1,5.440604
2,5.426804
3,5.387159
4,5.360421
...,...
73,5.017016
74,4.97483
75,4.930091
76,5.023079


In [ ]:
seoul_data = seoul_data.astype(float)
seoul_data

,0
0,5.417111
1,5.440604
2,5.426804
3,5.387159
4,5.360421
...,...
73,5.017016
74,4.974830
75,4.930091
76,5.023079


In [ ]:
len(pp_date) == len(seoul_data)

True

In [ ]:
pp_df = pd.DataFrame({'date': pp_date, 'pp_rate': seoul_data})
pp_df.iloc[72]

,72
date,2024-05
pp_rate,5.091858


In [ ]:
merged_pp_df = pd.merge(merged_rate_df, pp_df, left_on='계약년월', right_on='date', how='left')

merged_pp_df = merged_pp_df.drop(columns=['date'])  # 'date' 컬럼 제거
merged_pp_df

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,"18,000",0,3,2018.0,상원6나길 24,-,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,"23,000",0,4,2019.0,불광로14길 4,-,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769
2,서울특별시 마포구 상암동,사보이시티디엠씨,전세,21.6500,2019-04,30,"15,500",0,10,2016.0,,-,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769
3,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,"11,000",0,4,1999.0,능동로19길 47,-,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769
4,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,"1,000",50,4,2005.0,도봉로 352,-,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272277,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,"60,900",25,5,2013.0,위례광장로 121,202405~202605,갱신,-,"60,900",0,아파트,2636.52,3.50,5.091858
3272278,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,"92,500",0,6,2015.0,위례광장로 185,202407~202607,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858
3272279,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,"5,000",179,19,2008.0,올림픽로 135,202406~202606,갱신,사용,"5,000",170,아파트,2636.52,3.50,5.091858
3272280,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,"100,000",0,2,2008.0,올림픽로 99,202407~202607,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858


#### 도로명 처리
- 도로명 `isnull()` 함수로 잡히지 않는 빈문자열 처리
- 시군구에서 동을 제거한 문자열과 도로명을 합쳐 전체주소 컬럼 작성

In [ ]:
# 빈 문자열인지 알아내는 함수
def isempty(series):
    """
    series parameter: pandas Series
    return: empty index list or -1 if dtype is not object
    """
    if series.dtype != 'O':  # 'O'는 object 타입을 의미 (문자열 등)
        return -1
    else:

      result = []
      a = list(series)
      for i in range(len(a)):
          if isinstance(a[i], str) and a[i].strip() == '':  # 문자열인지 확인 후 체크
              result.append(i)

      return result if result else -1  # 빈 리스트면 -1 반환

result = isempty(merged_pp_df['도로명'])
total_df_cleaned = merged_pp_df.drop(result)

In [ ]:
# 문자열인 컬럼들의 빈 문자열들 확인
col = ['시군구', '월세금(만원)', '주택유형','계약년월','전월세구분']
for i in col:
  if isempty(merged_pp_df[i]) == -1:
    print(f"{i}: 빈 문자열 없음")
  else:
    print(f"{i}: {isempty(merged_pp_df[i])}")

시군구: 빈 문자열 없음
월세금(만원): 빈 문자열 없음
주택유형: 빈 문자열 없음
계약년월: 빈 문자열 없음
전월세구분: 빈 문자열 없음


In [ ]:
# 빈 문자열 또는 공백만 있는 행들 제거
result = isempty(merged_pp_df['도로명'])
total_df_cleaned = merged_pp_df.drop(result)

In [ ]:
total_df_cleaned['도로명']
total_df_cleaned.reset_index(drop=True, inplace=True)

#### 전체 주소 컬럼 작성

In [ ]:
# '시군구'에서 마지막 띄어쓰기 기준으로 동 제거
total_df_cleaned['시군구_수정'] = total_df_cleaned['시군구'].apply(lambda x: ' '.join(x.split()[:-1]))

# '도로명'과 결합하여 '전체주소' 생성
total_df_cleaned['전체주소'] = total_df_cleaned['시군구_수정'] + ' ' + total_df_cleaned['도로명']


In [ ]:
total_df_cleaned[['도로명','시군구_수정','전체주소']]

,도로명,시군구_수정,전체주소
0,상원6나길 24,서울특별시 성동구,서울특별시 성동구 상원6나길 24
1,불광로14길 4,서울특별시 은평구,서울특별시 은평구 불광로14길 4
2,능동로19길 47,서울특별시 광진구,서울특별시 광진구 능동로19길 47
3,도봉로 352,서울특별시 강북구,서울특별시 강북구 도봉로 352
4,새창로 213-12,서울특별시 용산구,서울특별시 용산구 새창로 213-12
...,...,...,...
3194709,위례광장로 121,서울특별시 송파구,서울특별시 송파구 위례광장로 121
3194710,위례광장로 185,서울특별시 송파구,서울특별시 송파구 위례광장로 185
3194711,올림픽로 135,서울특별시 송파구,서울특별시 송파구 올림픽로 135
3194712,올림픽로 99,서울특별시 송파구,서울특별시 송파구 올림픽로 99


In [ ]:
total_df_cleaned

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,...,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,시군구_수정,전체주소
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,"18,000",0,3,2018.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 성동구,서울특별시 성동구 상원6나길 24
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,"23,000",0,4,2019.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 은평구,서울특별시 은평구 불광로14길 4
2,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,"11,000",0,4,1999.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 광진구,서울특별시 광진구 능동로19길 47
3,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,"1,000",50,4,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 강북구,서울특별시 강북구 도봉로 352
4,서울특별시 용산구 한강로2가,한강현대하이엘,전세,40.2800,2019-04,30,"21,000",0,15,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 용산구,서울특별시 용산구 새창로 213-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194709,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,"60,900",25,5,2013.0,...,갱신,-,"60,900",0,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 121
3194710,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,"92,500",0,6,2015.0,...,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 185
3194711,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,"5,000",179,19,2008.0,...,갱신,사용,"5,000",170,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 135
3194712,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,"100,000",0,2,2008.0,...,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 99


### 최종 데이터 확인 및 저장

#### 전월세 전환 데이터 생성 및 저장

In [ ]:
# change_df = total_df_cleaned.copy()

In [ ]:
# # 보증금과 월세를 숫자로 변환
# change_df["보증금(만원)"] = change_df["보증금(만원)"].str.replace(",", "").astype(int)
# change_df["월세금(만원)"] = change_df["월세금(만원)"].str.replace(",", "").fillna(0).astype(int)

# # 월세 변환 조건 (전월세구분이 '월세'이고, 월세금이 0보다 큰 경우)
# mask = (change_df["월세금(만원)"] > 0) & (change_df["전월세구분"] == "월세")

# # 전세 보증금 변환 (pp_rate가 0이면 1로 대체)
# change_df.loc[mask, "보증금(만원)"] += (change_df.loc[mask, "월세금(만원)"] * 12) / change_df.loc[mask, "pp_rate"].replace(0, 1)
# change_df.loc[mask, "보증금(만원)"] = change_df.loc[mask, "보증금(만원)"].astype(int)  # 정수 변환


<ipython-input-72-8ee5bacf5b70>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 1113.51233846  2068.10740308  1170.2685077  ... 62470.70111081
 60958.91759234  5421.84996116]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  change_df.loc[mask, "보증금(만원)"] += (change_df.loc[mask, "월세금(만원)"] * 12) / change_df.loc[mask, "pp_rate"].replace(0, 1)


In [ ]:
change_df

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,...,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,시군구_수정,전체주소
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,18000.0,0,3,2018.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 성동구,서울특별시 성동구 상원6나길 24
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,23000.0,0,4,2019.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 은평구,서울특별시 은평구 불광로14길 4
2,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,11000.0,0,4,1999.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 광진구,서울특별시 광진구 능동로19길 47
3,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,1113.0,50,4,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 강북구,서울특별시 강북구 도봉로 352
4,서울특별시 용산구 한강로2가,한강현대하이엘,전세,40.2800,2019-04,30,21000.0,0,15,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 용산구,서울특별시 용산구 새창로 213-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194709,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,60958.0,25,5,2013.0,...,갱신,-,"60,900",0,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 121
3194710,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,92500.0,0,6,2015.0,...,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 185
3194711,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,5421.0,179,19,2008.0,...,갱신,사용,"5,000",170,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 135
3194712,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,100000.0,0,2,2008.0,...,신규,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 99


In [ ]:
# 월세 컬럼 제거
# change_df.drop(columns=["월세금(만원)"], inplace=True)

In [ ]:
# change_df.columns

Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '층', '건축년도',
       '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)', '종전계약 월세(만원)',
       '주택유형', 'index', 'rate', 'pp_rate', '시군구_수정', '전체주소'],
      dtype='object')

In [ ]:
# from google.colab import files


# change_df.reset_index(drop=True, inplace=True)
# change_df.to_csv("change_data.csv", index=False)
# files.download("change_data.csv")  # 파일 다운로드

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### 전월세 전환 하지 않고 전세만 남긴 데이터 저장

In [ ]:
from google.colab import files
full_df = total_df_cleaned.copy()

full_df.reset_index(drop=True, inplace=True)
full_df.to_csv("full_data.csv", index=False)
files.download("full_data.csv")  # 파일 다운로드

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>